In [1]:
!pip install gym3
import lib.util

In [4]:
# make module

from lib.masked_attention import MaskedAttention
#REQUIRES:
import functools
import torch as th
from torch import nn
import lib.xf as xf
from lib.minecraft_util import store_args
from lib.tree_util import tree_map

mask_attention_module = MaskedAttention(
        input_size=128,
        memory_size=256,
        heads=2,
        timesteps=128,
        mask = "clipped_causal",
        init_scale=1,
        norm="none",
        log_scope="sa",
        use_muP_factor=False)



In [ ]:
# put matrix through module

import torch as th

array = th.normal(0,1,(1000,128,16,16))
print(array.mean())
array2 = array[:,0,None,None]
print(array2.shape)
print(array2.mean())


tensor(-8.4675e-05)
torch.Size([1000, 1, 1, 16, 16])
tensor(0.0011)
